### Imports

In [1]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

### Load data

In [2]:
train = pd.read_csv("../../data/home_default/train.csv")
test  = pd.read_csv("../../data/home_default/test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)


### Merge train and test data

In [3]:
full = pd.concat([train, test])
train_N = len(train)

### Fill categorical NaN

In [4]:
for col in ('NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'):
    full[col] = full[col].fillna(full[col].mode()[0])

### Fill numeric NaN

In [5]:
missing_cols = [col for col in full.columns if any(full[col].isnull())]
for col in missing_cols:
    full[col] = full[col].fillna(full[col].median())

### Confirm missing values are gone

In [6]:
full.isnull().sum().sum()

0

### Get dummies

In [7]:
full = pd.get_dummies(full)

### Separate data back into train and test

In [8]:
train = full[:train_N]
test = full[train_N:]
del full, train_N

### Split again into predictors, target, and id

In [9]:
train_y = train.TARGET
train_x = train.drop(["TARGET", "SK_ID_CURR"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test.drop(["TARGET", "SK_ID_CURR"], axis=1)

# Modeling

### Imports and scoring helper function

In [10]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=2)
    
def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, cv=kfold, scoring="neg_mean_squared_error"))
    print(rmse)
    print()
    print(sum(rmse) / len(rmse))

## LGBM Regressor

(open Markdown for notes on [LGBM](http://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.LGBMRegressor) and parameter search attempts)

<div hidden>

# LGBM Parameters

boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, **kwargs

<\div>

In [11]:
lgbm_model = LGBMRegressor()

start = time.time()
lgbm_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 15 seconds


### Scoring

In [12]:
start = time.time()
rmsle_cv(lgbm_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26223286  0.26024019]

0.261236525624

Scoring took 19 seconds


## Linear Regressor

(open Markdown for notes on [Linear Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and parameter search attempts)

<div hidden>

# Linear Parameters

fit_intercept=True,
normalize=False,
copy_X=True,
n_jobs=1

<\div>

In [13]:
lin_model = LinearRegression()

start = time.time()
lin_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 7 seconds


### Scoring

In [14]:
start = time.time()
rmsle_cv(lin_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26508292  0.26264828]

0.263865596742

Scoring took 7 seconds


## Random Forest Regressor

WARNING: Takes a lot of time

In [24]:
rf_model = RandomForestRegressor()

start = time.time()
rf_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

[ 0.27978732  0.28013387  0.28058117]

0.280167454734


### Scoring

In [ ]:
start = time.time()
rmsle_cv(rf_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Logistic Regression

(open Markdown for notes on [Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and parameter search attempts)

<div hidden>

# Logistic Parameters

penalty=’l2’,
dual=False,
tol=0.0001,
C=1.0,
fit_intercept=True,
intercept_scaling=1,
class_weight=None,
random_state=None,
solver=’liblinear’,
max_iter=100,
multi_class=’ovr’,
verbose=0,
warm_start=False,
n_jobs=1

<\div>

In [15]:
log_model = LogisticRegression()

start = time.time()
log_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 15 seconds


### Scoring

In [16]:
start = time.time()
rmsle_cv(log_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.28561741  0.2826656 ]

0.2841415038

Scoring took 18 seconds


## Lasso Regression

(open Markdown for notes on [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) and parameter search attempts)

<div hidden>

# Lasso Parameters

alpha=1.0,
fit_intercept=True,
normalize=False,
precompute=False,
copy_X=True,
max_iter=1000,
tol=0.0001,
warm_start=False,
positive=False,
random_state=None,
selection=’cyclic’

<\div>

In [17]:
las_model = Lasso(random_state=17)

start = time.time()
las_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 68 seconds


### Scoring

In [18]:
start = time.time()
rmsle_cv(las_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.27209767  0.26929769]

0.270697679661

Scoring took 53 seconds


## Ridge Regression


(open Markdown for notes on [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) and parameter search attempts)

<div hidden>

# Ridge Parameters

alpha=1.0,
fit_intercept=True,
normalize=False,
copy_X=True,
max_iter=None,
tol=0.001,
solver=’auto’,
random_state=None

<\div>

In [20]:
rid_model = Ridge(random_state=17)

start = time.time()
rid_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 3 seconds


### Scoring

In [21]:
start = time.time()
rmsle_cv(rid_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26507705  0.26264958]

0.263863313366

Scoring took 3 seconds


## XGB Regressor

In [25]:
xgb_model = XGBRegressor()

start = time.time()
xgb_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

NameError: name 'score_model' is not defined

### Scoring

Scoring takes 349 seconds for me

In [26]:
start = time.time()
rmsle_cv(xgb_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26315737  0.26094917]

0.262053265711

Scoring took 349 seconds


### Get predictions

In [27]:
predictions = xgb_model.predict(test_x)

### Restrict predictions to appropriate range

In [28]:
predictions = np.clip(predictions, 0, 1)

# Sanity check
any(predictions < 0) or any(predictions > 1)

False

### Save file to CSV

In [29]:
pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": predictions
}).to_csv("../../submissions/xgb.csv", index=False)